In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load and prepare the dataset
with open("D:/Basudev/genaiprereq/Day2/poetry.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences for training
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Split into features and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the RNN model
model = Sequential([
    Embedding(total_words, 64, input_length=max_sequence_len - 1),
    SimpleRNN(150, return_sequences=True),
    SimpleRNN(100),
    Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X, y, epochs=100, verbose=1)

# Text generation function
def generate_poetry(seed_text, next_words=20):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        next_word = tokenizer.index_word[np.argmax(predicted)]
        seed_text += " " + next_word
    return seed_text

# Generate a poem with a seed text
print(generate_poetry("In the midst of silence"))


Epoch 1/100
6/6 [==============================] - 2s 7ms/step - loss: 4.9102 - accuracy: 0.0111
Epoch 2/100
6/6 [==============================] - 0s 7ms/step - loss: 4.7140 - accuracy: 0.0611
Epoch 3/100
6/6 [==============================] - 0s 6ms/step - loss: 4.5751 - accuracy: 0.1000
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 4.4627 - accuracy: 0.1167
Epoch 5/100
6/6 [==============================] - 0s 6ms/step - loss: 4.3421 - accuracy: 0.1556
Epoch 6/100
6/6 [==============================] - 0s 7ms/step - loss: 4.2195 - accuracy: 0.1556
Epoch 7/100
6/6 [==============================] - 0s 3ms/step - loss: 4.0938 - accuracy: 0.1833
Epoch 8/100
6/6 [==============================] - 0s 6ms/step - loss: 3.9418 - accuracy: 0.2333
Epoch 9/100
6/6 [==============================] - 0s 7ms/step - loss: 3.7994 - accuracy: 0.2389
Epoch 10/100
6/6 [==============================] - 0s 6ms/step - loss: 3.6288 - accuracy: 0.3333
Epoch 11/100
6/6 [===========